# Decision Tree Classifier - From Scratch


## Importing required Libraries and Data

In [50]:
import numpy as np

data = [
    [12.0, 1.5, 1, 'Wine'],
    [5.0, 2.0, 0, 'Beer'],
    [40.0, 0.0, 1, 'Whiskey'],
    [13.5, 1.2, 1, 'Wine'],
    [4.5, 1.8, 0, 'Beer'],
    [38.0, 0.1, 1, 'Whiskey'],
    [11.5, 1.7, 1, 'Wine'],
    [5.5, 2.3, 0, 'Beer']
]

##Data pre - processing and cleaning

In [51]:
encode = {'Wine' :0, 'Beer' : 1, 'Whiskey':2}
decode = {v: k for k, v in encode.items()}
data = np.array(data)
data[:, 3] = [encode[types] for types in data[:, 3]]
data = data.astype(float)

X = data[:,:3]
y = data[:,3]
data

array([[12. ,  1.5,  1. ,  0. ],
       [ 5. ,  2. ,  0. ,  1. ],
       [40. ,  0. ,  1. ,  2. ],
       [13.5,  1.2,  1. ,  0. ],
       [ 4.5,  1.8,  0. ,  1. ],
       [38. ,  0.1,  1. ,  2. ],
       [11.5,  1.7,  1. ,  0. ],
       [ 5.5,  2.3,  0. ,  1. ]])

## Implementing Gini Function

In [52]:
def gini_impurity(y):
    total = len(y)
    class_counts = {}
    for label in y:
        label = int(label)
        if label not in class_counts:
            class_counts[label] = 0
        class_counts[label] += 1

    impurity = 1.0
    for count in class_counts.values():
        prob = count / total
        impurity -= prob ** 2
    return impurity

def majority_class(y):
    class_counts = {}
    for label in y:
        label = int(label)
        if label not in class_counts:
            class_counts[label] = 0
        class_counts[label] += 1

    max_count = -1
    majority_label = None
    for label, count in class_counts.items():
        if count > max_count:
            max_count = count
            majority_label = label
    return majority_label

##Implementing the best fit finder


In [53]:
def best_split(X, y):
    best_feature, best_thresh, best_gini = None, None, float('inf')
    n_samples, n_features = X.shape

    for feature_index in range(n_features):
        thresholds = np.unique(X[:, feature_index])
        for threshold in thresholds:
            left_indices = X[:, feature_index] <= threshold
            right_indices = X[:, feature_index] > threshold

            if np.sum(left_indices) == 0 or np.sum(right_indices) == 0:
                continue

            gini_left = gini_impurity(y[left_indices])
            gini_right = gini_impurity(y[right_indices])
            weighted_gini = (len(y[left_indices]) * gini_left + len(y[right_indices]) * gini_right) / len(y)

            if weighted_gini < best_gini:
                best_gini = weighted_gini
                best_feature = feature_index
                best_thresh = threshold

    return best_feature, best_thresh

##Implementing recurssive Tree building


In [54]:
class Node:
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, value=None):
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

def build_tree(X, y, depth=0, max_depth=3, min_samples=1):
    if len(set(y)) == 1 or depth >= max_depth or len(y) < min_samples:
        return Node(value=majority_class(y))

    feature_index, threshold = best_split(X, y)
    if feature_index is None:
        return Node(value=majority_class(y))

    left_indices = X[:, feature_index] <= threshold
    right_indices = X[:, feature_index] > threshold

    left = build_tree(X[left_indices], y[left_indices], depth + 1, max_depth, min_samples)
    right = build_tree(X[right_indices], y[right_indices], depth + 1, max_depth, min_samples)

    return Node(feature_index, threshold, left, right)


##Prediction


In [55]:
def predict_sample(x, node):
    if node.value is not None:
        return node.value
    if x[node.feature_index] <= node.threshold:
        return predict_sample(x, node.left)
    else:
        return predict_sample(x, node.right)

def predict(X_test, tree):
    return [predict_sample(x, tree) for x in X_test]


##Evaluation

In [57]:
tree = build_tree(X, y)
test_data = np.array([
    [6.0, 2.1, 0],   # Expected: Beer
    [39.0, 0.05, 1], # Expected: Whiskey
    [13.0, 1.3, 1]   # Expected: Wine
])

preds = predict(test_data, tree)
predicted_labels = [decode[p] for p in preds]

print("Predicted Labels:", predicted_labels)

Predicted Labels: ['Wine', 'Whiskey', 'Wine']
